# 变分自动编码器

参考资料：

- [Understanding Variational Autoencoders (VAEs)](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)
- [L1aoXingyu/pytorch-beginner/08-AutoEncoder/Variational_autoencoder.py](https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/Variational_autoencoder.py)
- [An Introduction to Autoencoders: Everything You Need to Know ](https://www.v7labs.com/blog/autoencoders-guide)
- [一文理解变分自编码器（VAE）](https://zhuanlan.zhihu.com/p/64485020)

在过去的几年中，由于其取得的一些惊人进步，基于深度学习的生成模型越来越受到关注。依赖于海量数据、精心设计的网络架构和智能训练技术，深度生成模型显示出令人难以置信的能力，可以生成各种高度逼真的内容，例如图像、文本和声音。在这些深度生成模型中，有两大家族脱颖而出并值得特别关注：生成对抗网络 (GAN) 和变分自编码器 (VAE)。

上一节文中讨论了生成对抗网络 (GAN)，并特别展示了如何训练两个网络——生成器和判别器，推动它们的改进迭代。现在在这篇文章中介绍另一种主要的深度生成模型：变分自编码器 (VAE)。简而言之，VAE 是一种自编码器，其编码分布 在训练期间被正则化，以确保其潜在空间具有良好的特性，允许我们生成一些新数据。此外，术语“变分”来自统计中正则化和变分推理方法之间的密切关系。

那么什么是自编码器？什么是潜在空间，为什么要对其进行正则化？如何从 VAE 生成新数据？ VAE 和变分推理之间有什么联系？这就是本文主要翻译的内容，以帮助我们构建关于VAE的基本概念，并且逐步构建从一开始就得出导致这些概念的推理，获取基本直觉。

在第一部分中，回顾一些关于降维和自动编码器的重要概念，它们对理解 VAE 很有用。然后，在第二部分，展示为什么不能使用自动编码器来生成新数据，并将介绍变分自动编码器，它是自动编码器的正则化版本，使生成过程成为可能。在这篇文章中，使用以下符号：对于随机变量 z，我们将用 p(z) 表示这个随机变量的分布（或密度，取决于上下文）。

## 降维、PCA 和自动编码器

在第一部分中，首先讨论一些与降维相关的概念。特别是，简要回顾主成分分析 (PCA) 和自动编码器，展示这两种想法如何相互关联。

### 什么是降维？

在机器学习中，降维是减少描述某些数据的特征数量的过程。这种减少是通过选择（仅保留一些现有特征）或通过提取（基于旧特征创建数量减少的新特征）来完成的，能用于需要低维数据（数据可视化、数据存储，繁重的计算……）的应用。尽管存在许多不同的降维方法，但我们可以设置一个与这些方法中的大多数匹配的全局框架。

首先，我们将编码器称为从“旧特征”表示（通过选择或提取）产生“新特征”表示的过程，并将解码器称为相反的过程。降维可以解释为数据压缩，其中编码器压缩数据（从初始空间到编码空间，也称为潜在空间），而解码器对它们进行解压缩。当然，根据初始数据分布、潜在空间维度和编码器定义，这种压缩可能是有损的，这意味着部分信息在编码过程中丢失，在解码时无法恢复。

![](img/1_UdOybs9wOe3zW8vDAfj9VA@2x.png)

降维方法的主要目的是在给定的系列中找到最佳的编码器/解码器对。换句话说，对于给定的一组可能的编码器和解码器，我们正在寻找在编码时保持最大信息量并且因此在解码时具有最小重构误差的对。如果我们分别将 E 和 D 表示我们正在考虑的编码器和解码器的家族，那么降维问题可以写成

![](img/1_9_DFaRan_hX9xMZldVGNjg@2x.png)

其中，$\epsilon (x,d(e(x)))$定义输入数据 x 和编码解码数据 d(e(x)) 之间的重构误差度量。最后请注意，在下文中，我们用 N 表示数据数量，$n_d$ 表示初始（解码）空间的维度，$n_e$ 表示缩减（编码）空间的维度。

### 主成分分析 (PCA)

在谈到降维时，首先想到的方法之一是[主成分分析 (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)。为了展示它如何适合我们刚刚描述的框架并建立其和自动编码器的联系，这里从较宏观的视角来看看PCA的原理。

PCA 的思想是构建 $n_e$ 个新的独立特征，这些特征是 $n_d$ 个旧特征的线性组合，使数据在这些新特征定义的子空间上的投影尽可能接近初始数据（以欧几里得距离）。换句话说，PCA 正在寻找初始空间的最佳线性子空间（由新特征的正交基描述），以便通过它们在该子空间上的投影来逼近数据，使误差尽可能小。

![](img/1_ayo0n2zq_gy7VERYmp4lrA@2x.png)

在前面说的全局框架中可以这么描述：我们正在寻找 $n_e$ 乘 $n_d$ 矩阵族 E 中的编码器（线性变换），其行是正交的（特征独立），并在 $n_d$ 乘 $n_e$ 矩阵族 D 中寻找相关的解码器。可以证明，对应于 协方差特征矩阵的 $n_e$ 个最大特征值 的 特征向量 是正交的（或可以选择正交），并且定义了 维度 $n_e$ 的最佳子空间以将数据投影到最小误差作为近似。因此，可以选择这 n_e 个特征向量作为我们的新特征，因此，降维问题可以表示为特征值/特征向量问题。此外，还可以证明，在这种情况下，解码器矩阵是编码器矩阵的转置。

![](img/1_LRPyMAwDlio7f1_YKYI2hw@2x.png)

### 自编码器

现在讨论自编码器，看看如何使用神经网络进行降维。自动编码器的一般思想非常简单，包括：将编码器和解码器设置为神经网络，并使用迭代优化过程学习最佳编码-解码方案。因此，在每次迭代中，我们将一些数据提供给自动编码器架构（编码器后跟解码器），我们将编码-解码的输出与初始数据进行比较，并通过架构反向传播误差以更新网络的权重。

因此，直观地说，整个自动编码器架构（编码器+解码器）为数据创建了一个瓶颈bottleneck，以确保只有信息的主要结构化部分可以通过并被重构。回到我们的一般框架：编码器族 E 由编码器网络架构定义，解码器族 D 由解码器网络架构定义，并且通过梯度下降来搜索最小化重构误差的编码器和解码器这些网络的参数。

![](img/1_bY_ShNK6lBCQ3D9LYIfwJg@2x.png)

让我们首先假设我们的编码器和解码器架构都只有一层且没有非线性（线性自动编码器）。这样的编码器和解码器是简单的线性变换，可以表示为矩阵。在这种情况下，我们可以看到与 PCA 的清晰联系，就像 PCA 一样，我们正在寻找最佳线性子空间来投影数据，同时尽可能减少信息丢失。使用 PCA 获得的编码和解码矩阵自然定义了我们可以通过梯度下降达到的解决方案之一，但我们应该知道这不是唯一的解决方案。实际上，可以选择多个基来描述相同的最佳子空间，因此，多个编码器/解码器对可以给出最佳重构误差。此外，对于线性自编码器，与 PCA 相反，我们最终得到的新特征不必是独立的（神经网络中没有正交性约束）。

![](img/1_ek9ZFmimq9Sr1sG5Z0jXfQ@2x.png)

现在，让我们假设编码器和解码器都是深度和非线性的。在这种情况下，架构越复杂，自编码器就越能在保持低重构损失的同时进行降维。直观地说，如果我们的编码器和解码器有足够的自由度，我们可以将任何初始维数减少到 1。实际上，具有“无穷功率”的编码器理论上可以将我们的 N 个初始数据点编码为 1、2、3， ... 最多 N （或更一般地说，作为实轴上的 N 整数）个，且相关的解码器可以进行反向转换，而在此过程中没有损失。

然而，在这里，我们应该记住两件事。首先，没有重建损失的重要降维通常伴随着代价：潜在空间中缺乏可解释和可利用的结构（缺乏规律性）。其次，在大多数情况下，降维的最终目的不仅仅是减少数据的维数，而是减少维数的同时将主要部分的数据结构信息保留在减少的表示中。由于这两个原因，必须根据降维的最终目的仔细控制和调整潜在空间的维度和自动编码器的“深度”（定义压缩的程度和质量）。

![](img/1_F-3zbCL_lp7EclKowfowMA@2x.png)

## 变分自编码器

到目前为止，我们已经讨论了降维问题并介绍了自动编码器，这些自动编码器是可以通过梯度下降训练的编码器-解码器架构。现在让我们将这些内容与生成问题联系起来，看看当前形式的自动编码器在这个问题上的局限，进而介绍变分自动编码器。

### 用于内容生成的自动编码器的局限性

在这一点上，一个很自然的问题是“自动编码器和内容生成之间的联系是什么？”。事实上，一旦自动编码器经过训练，我们就有了编码器和解码器，但仍然没有真正的方法来产生任何新内容。乍一看，我们可能会认为，如果潜在空间足够规则（在训练过程中由编码器“组织得很好”），我们可以从潜在空间中随机抽取一个点并对其进行解码以得到一个新的内容。然后，解码器就或多或少地像生成对抗网络的生成器一样。

![](img/1_Qd1xKV9o-AnWtfIDhhNdFg@2x.png)

然而，正如我们在上一节中讨论的，自动编码器的潜在空间的规律性是一个难点，它取决于初始空间中数据的分布、潜在空间的维度和编码器的架构。因此，先验地确保编码器以与我们刚刚描述的生成过程兼容的智能方式组织潜在空间是非常困难的。

为了说明这一点，让我们考虑我们之前给出的示例，其中我们描述了一个编码器和一个解码器，其功能强大到足以将任何 N 个初始训练数据放在实轴上（每个数据点都被编码为一个实数）并在没有任何损失情况下对它们进行解码重建。在这种情况下，自动编码器的高度自由使得编码和解码不会丢失信息（尽管潜在空间的维数低）而导致严重的过度拟合，这意味着一旦解码潜在空间的某些点将会给出无意义的内容。这个一维例子被选择得比较极端，自编码器潜在空间规律性的问题比这更普遍，值得特别关注。

![](img/1_iSfaVxcGi_ELkKgAG0YRlQ@2x.png)

仔细想一想，编码数据到潜在空间之间缺乏结构是很正常的。事实上，自动编码器在任务中并没有被训练来强制获得这样的组织：自动编码器只是被训练以尽可能少的损失进行编码和解码，无不论潜在空间是如何组织的。因此，如果我们不小心架构的定义，很自然地，在训练期间，网络会利用任何过度拟合的可能性来尽可能地完成其任务，除非我们明确地对其进行正则化！

### 变分自编码器的定义

因此，为了能够将自动编码器的解码器用于生成目的，我们必须确保潜在空间足够规则。获得这种规律性的一种可能解决方案是在训练过程中引入显式正则化。因此，正如我们在这篇文章的介绍中简要提到的，变分自编码器可以定义为这样一种自编码器，其训练被正则化以避免过度拟合并确保潜在空间具有良好的特性以支持生成过程。

就像标准的自动编码器一样，变分自动编码器是一种由编码器和解码器组成的架构，经过训练以最小化编码解码数据和初始数据之间的重构误差。然而，为了引入潜在空间的一些正则化，我们对编码-解码过程进行了轻微修改：我们不是将输入编码为单个点，而是将其编码为潜在空间上的分布。然后模型训练如下：

1. 首先，输入被编码为潜在空间上的分布
2. 其次，潜在空间中的一个点被从该分布中采样
3. 第三，对采样点进行解码，计算重构误差
4. 最后，重构误差通过网络反向传播

![](img/1_ejNnusxYrn1NRDZf4Kg2lw@2x.png)

在实践中，编码分布被选择为正态分布，以便可以训练编码器返回描述这些高斯分布的均值和协方差矩阵。将输入编码为具有某种方差的分布而不是单个点的原因是，它可以非常自然地表达潜在空间正则化：编码器返回的分布强制接近标准正态分布。我们将在下一小节中看到，我们以这种方式确保了潜在空间的局部和全局正则化（局部因为方差控制，全局因为均值控制）。
因此，在训练 VAE 时最小化的损失函数由一个“重构项”（在最后一层）和一个“正则化项”（在潜在层）构成，它倾向于通过使编码器返回的分布接近标准正态分布来规范潜在空间的组织。该正则化项表示为返回分布与标准高斯分布之间的 Kulback-Leibler 散度，并将在下一节中进一步证明。我们可以注意到，两个高斯分布之间的 Kullback-Leibler 散度具有封闭形式，可以直接用两个分布的均值和协方差矩阵表示。

![](img/1_Q5dogodt3wzKKktE0v3dMQ@2x.png)

### 关于正则化的直觉

为了使生成过程成为可能，潜在空间所期望的规律性可以通过两个主要属性来表达：连续性（潜在空间中的两个接近点在解码后不应给出两个完全不同的内容）和完整性（对于选定的分布，从潜在空间中采样的点应该在解码后给出“有意义”的内容）。

![](img/1_83S0T8IEJyudR_I5rI9now@2x.png)

VAE 将输入编码为分布而不是简单点的这样一条规则并不足以确保连续性和完整性。如果没有明确定义的正则化项，模型仍可以学习，以最小化其重构误差，它会“忽略”返回分布这一事实，会表现得几乎像经典自动编码器（导致过度拟合）。为此，编码器可以返回具有微小方差的分布或返回具有非常不同均值的分布（在潜在空间中彼此相距很远）。在这两种情况下，分布都以错误的方式使用（和我们的预期是不符的）并且不满足连续性和/或完整性。

因此，为了避免这些影响，我们必须对协方差矩阵和编码器返回的分布均值进行正则化。在实践中，这种正则化是通过强制分布接近标准正态分布（中心和减少）来完成的。这样，我们要求协方差矩阵接近恒等式，防止方差太小，并且均值接近 0，防止编码分布彼此相距太远。

![](img/1_9ouOKh2w-b3NNOVx4Mw9bg@2x.png)

有了这个正则化项，我们可以防止模型在潜在空间中对相距很远的数据进行编码，并鼓励尽可能多的返回分布“重叠”，以这种方式满足预期的连续性和完整性条件。自然，对于任何正则化项，其代价是训练数据的重建误差更高。然而，重建误差和 KL 散度之间的权衡可以调整，我们将在下一节看到平衡的表达如何从我们的形式推导中自然出现。

总结本小节，我们可以观察到通过正则化获得的连续性和完整性倾向于在潜在空间中编码的信息上创建“梯度”。例如，介于来自不同训练数据的两个编码分布的均值之间的潜在空间上的点应该被解码到介于给出第一个分布的数据和给出第二个分布的数据之间，因为在这两种情况下它都可能被自动编码器采样。

![](img/1_79AzftDm7WcQ9OfRH5Y-6g@2x.png)

## VAE相关的数学

关于VAE为什么叫VAE，它和变分推理之间的关系，因为比较多的数学公式，这里暂时没翻译，更多内容可以查看下面的资料：

- [Understanding Variational Autoencoders (VAEs)](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)
- [Bayesian inference problem, MCMC and variational inference](https://towardsdatascience.com/bayesian-inference-problem-mcmc-and-variational-inference-25a8aa9bce29)
- [一文理解变分自编码器（VAE）](https://zhuanlan.zhihu.com/p/64485020)

这里主要简单描述下里面的数学逻辑。

如前所述，AE就像下面这样，潜变量不是概率形式的

![](img/60bbe752a4dbfa4ee8b4912e_funny1.png)

而VAE则是以概率分布的方式表示潜变量，所以就有连续的潜变量空间，我们可以去采样。VAE像下面这样。

![](img/60bbe7e0e2a8da8a3a877a05_pasted1.png)

然后潜变量值是从分布中采样来重构输入。

表示为概率分布的过程是这样的：

我们的目标是识别潜在向量 z 的特征，该特征在给定特定输入的情况下重构输出。实际上，我们想研究给定输出 x的潜在向量的特征，即z的后验分布[p(z|x)] 。

虽然在数学上直接给出分布不可能，但有一个更简单、更容易的方法，就是构建一个可以为我们估计分布的参数化模型，通过最小化原始分布和我们参数化分布之间的 KL 散度来实现。

将参数化分布表示为 q，我们可以推断出图像重建中可能使用的潜在属性。

假设先验 z 是一个多元高斯模型，我们可以构建一个参数化分布——包含均值和方差两个参数的分布。然后对相应的分布进行采样并馈送到解码器，然后解码器从采样点重建输入。

虽然这在理论上看起来很容易，但因为在将数据馈送到解码器之前执行的随机采样过程导致无法定义反向传播使得它变得不可能实现。

为了克服这个障碍，我们使用了再参数化技巧——一种巧妙定义的方法来绕过神经网络的采样过程。

这是什么一回事呢？

在重新参数化技巧中，我们从单位高斯随机采样一个值ε，然后通过潜在分布方差σ对其进行缩放并将其移动相同的均值μ。

现在，我们已经将采样过程作为在反向传播管道处理之外完成的工作，并且采样值 ε 就像模型的另一个输入一样，在瓶颈处输入。

我们所获得的结果的示意图可以表示为：

![](img/60e424b06f61a263edba1fe6_diagrammetic.png)

因此，变分自编码器允许我们学习输入数据的平滑潜在状态表示。

为了训练 VAE，我们使用两个损失函数：重建损失和 KL 散度。

虽然重建损失使分布能够正确描述输入，但通过仅关注最小化重建损失，网络学习了非常窄的分布——类似于离散的潜在属性。

KL 散度损失阻止网络学习窄分布，并试图使分布更接近单位正态分布。

总结的损失函数可以表示为：

![](img/60bbe8a34dedb54fc00b7e0f_N denotes.png)

变分自编码器的主要用途可以在生成建模中看到。

从训练的潜在分布中采样并将结果馈送到解码器可以导致在自动编码器中生成数据。

通过训练变分自编码器生成的 MNIST 数字示例如下所示：

![](img/60bbe8d0e8d8f0c41509914f_minstdigits.png)

小结这篇文章的主要内容：

- 降维是减少描述某些数据的特征数量的过程（通过仅选择初始特征的一个子集或通过将它们组合成数量减少的新特征），因此，可以看作是一个编码过程
- 自动编码器是由编码器和解码器组成的神经网络架构，它们为数据创建了一个瓶颈，并被训练在编码-解码过程中丢失最少的信息量（通过梯度下降迭代训练，目标是减少重建误差）
- 由于过度拟合，自动编码器的潜在空间可能非常不规则（潜在空间中的近点可以提供非常不同的解码数据，潜在空间的某些点一旦解码就可以提供无意义的内容，......），因此，我们不能真正定义一个生成过程，该过程仅包括从潜在空间中采样一个点并使其通过解码器以获取新数据
- 变分自编码器 (VAE) 是一种自编码器，它通过使编码器返回潜在空间上的分布而不是单个点，并通过在损失函数中添加该返回分布的正则化项来解决潜在空间不规则性问题，以确保更好地组织潜在空间
- 假设一个简单的潜在概率模型来描述我们的数据，可以仔细推导出 VAE 的非常直观的损失函数，由重构项和正则化项组成，特别是使用变分推理的统计技术（因此得名“变分”自编码器）

总而言之，我们可以概述在过去几年中，GAN 比 VAE 受益于更多的科学贡献。除其他原因外，与统治 GAN 的对抗性训练概念的简单性相比，VAE 理论基础（概率模型和变分推理）的复杂程度更高，可以部分解释为什么社区对 GAN 表现出的更高兴趣。通过这篇文章，我们希望我们能够分享宝贵的直觉和强大的理论基础，使新人更容易使用 VAE。

In [ ]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [ ]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data[0] / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')